This notebook plots SEDs and the corresponding photometry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from matplotlib.ticker import MaxNLocator
from moviepy.editor import ImageSequenceClip
from glob import glob
from pzflow.examples import get_galaxy_data

In [ ]:
# load the filters
filters = {}
for band in "ugrizy":
    x, y = np.genfromtxt(f"data/{band}_bandpass.dat", unpack=True)
    x *= 10
    y *= x
    y /= np.trapz(y, x)
    filters[band] = np.array([x, y])

# calculate the effective wavelengths
filter_wavelen = np.array([np.trapz(x * y, x) for (x, y) in filters.values()])

In [ ]:
def plot_filters(ax):
    # get the span and orientation of the plot
    ymin, ymax = ax.get_ylim()
    span = np.abs(ymax - ymin)
    scale = 1 if ymax > ymin else -1

    # loop through and plot the filters
    for (x, y) in filters.values():
        y_scaled = y / y.max() * 0.5 * span
        ax.fill_between(x, ymin, ymin + scale * y_scaled, color="silver", alpha=0.4)

In [ ]:
def load_sed(file, kernel=7):
    # load the SED from the file
    sed = np.genfromtxt(f"data/{file}.sed").T

    # cut off the SED above 12,000 Angstroms
    sed = sed[:, :np.abs(sed[0] - 12_000).argmin()]

    # rescale the SED so that magnitudes are realistic
    sed[1] *= 1e-16

    # smooth SED using the kernel
    sed[1] = medfilt(sed[1], kernel)

    # append the SED in AB magnitude
    with np.errstate(divide='ignore'):
        sed = np.vstack((sed, -2.5 * np.log10(9.1986 * sed[0]**2 * sed[1])))
    sed[2] = np.clip(sed[2], None, 30)

    return sed

In [ ]:
def redshift_sed(sed, z, dim=False):
    # stretch the wavelength grid
    redshifted_wavelen = (1 + z) * sed[0]

    # scale down the flux density
    if dim:
        redshifted_flambda = sed[1] / (1 + z)
    else:
        redshifted_flambda = sed[1]
        
    # interpolate on the original grid
    redshifted_flambda = np.interp(sed[0], redshifted_wavelen, redshifted_flambda)

    # recalculate magnitudes
    with np.errstate(divide='ignore'):
        redshifted_mags = -2.5 * np.log10(9.1986 * sed[0]**2 * redshifted_flambda)
    redshifted_mags = np.clip(redshifted_mags, None, 30)
    
    return np.array([sed[0], redshifted_flambda, redshifted_mags])

In [ ]:
def calculate_photometry(sed):
    fluxes = []
    for ((x, y), w) in zip(filters.values(), filter_wavelen):
        # interpolate the filter onto the SED grid
        y = np.interp(sed[0], x, y, left=0, right=0)

        # calculate the integrated flux
        fluxes.append(np.trapz(sed[1] * y, sed[0]))

    return np.array(fluxes)

# calculate the zero point flux for each band
flat_sed = np.linspace(100, 12_000, 1000)
flat_sed = np.array([flat_sed, 1 / (9.1986 * flat_sed**2)])
zero_points = calculate_photometry(flat_sed)

In [ ]:
def plot_photometry(ax, sed, mags=False):
    # calculate photometry
    photometry = calculate_photometry(sed)

    # calculate magnitudes?
    if mags:
        photometry = -2.5 * np.log10(photometry / zero_points)

    # plot the photometry
    ax.scatter(
        filter_wavelen,
        photometry,
        marker="*",
        s=100,
        lw=0.65,
        c="C1",
        edgecolors='k',
        zorder=10, 
    )

    return photometry

In [ ]:
El = load_sed("El_B2004a")
Sb = load_sed("SB2_B2004a")

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

plt.tight_layout()
fig.savefig("figures/restframe_sed.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# plot the redshifted SED
redshifted_sed = redshift_sed(Sb, 0.2)
ax.plot(redshifted_sed[0], redshifted_sed[1], c="C3")

# draw arrows indicating the redshift
ax.annotate(
    "",
    xy=(4400, 0.6e-15),
    xytext=(3800, 0.6e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)
ax.annotate(
    "",
    xy=(5975, 0.6e-15),
    xytext=(5075, 0.6e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)
ax.annotate(
    "",
    xy=(7850, 0.6e-15),
    xytext=(6625, 0.6e-15),
    arrowprops=dict(arrowstyle="->", color="k"),
)

# label the redshift
ax.text(0.97, 0.97, "z=0.25", ha="right", va="top", transform=ax.transAxes)

plt.tight_layout()
fig.savefig("figures/redshifted_sed.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(0, 1e-15),
    xlabel="Wavelength ($\AA$)",
    ylabel="Flux density",
)
ax.ticklabel_format(axis="y", useMathText=True)

# plot the original SED
ax.plot(Sb[0], Sb[1])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

# plot filters and photometry
plot_filters(ax)
plot_photometry(ax, Sb)

plt.tight_layout()
fig.savefig("figures/restframe_photometry.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax.set(
    xlim=(2_500, 11_000),
    ylim=(19.5, 15.75),
    xlabel="Wavelength ($\AA$)",
    ylabel="Magnitude",
)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

# plot the original SED
ax.plot(Sb[0], Sb[2])

# label the redshift
ax.text(0.97, 0.97, "z=0", ha="right", va="top", transform=ax.transAxes)

# plot filters and photometry
plot_filters(ax)
plot_photometry(ax, Sb, mags=True)
    
plt.tight_layout()
fig.savefig("figures/restframe_photometry_mags.png", dpi=500)

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)

# directory where individual frames are stored
frame_dir = "figures/sed_redshift"

# plot the redshifted SED
for i, redshift in enumerate(np.arange(0, 1.8, 0.05)):
    ax.clear()
    
    ax.set(
        xlim=(2_500, 11_000),
        ylim=(0, 1.5e-16),
        xlabel="Wavelength ($\AA$)",
        ylabel="Flux density",
    )
    ax.ticklabel_format(axis="y", useMathText=True)

    redshifted_sed = redshift_sed(El, redshift)
    ax.plot(redshifted_sed[0], redshifted_sed[1], c="C0")

    # label the redshift
    ax.text(0.97, 0.97, f"z={redshift:.2f}", ha="right", va="top", transform=ax.transAxes)

    # plot filters and photometry
    plot_filters(ax)
    plot_photometry(ax, redshifted_sed)

    # only set the tight layout on the first iteration
    if i == 0:
        plt.tight_layout()

    # save the frame
    fig.savefig(f"{frame_dir}/step{i}.png", dpi=500)

# get the list of frames
file_list = [f"{frame_dir}/step{i}.png" for i in range(len(glob(f"{frame_dir}/*")))]

# combine the frames into a video
clip = ImageSequenceClip(file_list, fps=4)
clip.write_gif("figures/sed_redshift.gif")


In [ ]:
fig, axes = plt.subplot_mosaic(
            """
            AABBB
            """,
            figsize=(6, 2.4), 
            dpi=150,
        )

# directory where individual frames are stored
frame_dir = "figures/color_redshift"

ug = []
gr = []

# plot the redshifted SED
for i, redshift in enumerate(np.arange(0, 1.8, 0.05)):

    axes["A"].clear()
    axes["B"].clear()

    axes["A"].set(
        ylabel="$u-g$", 
        xlabel="$g-r$",
        xlim=(-0.5, 2.5),
        ylim=(-0.5, 2.5),
    )
    axes["B"].set(
        ylabel="Magnitude", 
        xlabel="Wavelength ($\AA$)", 
        xlim=(2_500, 11_000), 
        ylim=(24.5, 16.5),
    )

    redshifted_sed = redshift_sed(El, redshift)
    axes["B"].plot(redshifted_sed[0], redshifted_sed[2], c="C0")

    # label the redshift
    axes["B"].text(0.03, 0.97, f"z={redshift:.2f}", ha="left", va="top", transform=axes["B"].transAxes)

    # plot filters and photometry
    plot_filters(axes["B"])
    photometry = plot_photometry(axes["B"], redshifted_sed, mags=True)

    ug.append(photometry[0] - photometry[1])
    gr.append(photometry[1] - photometry[2])
    axes["A"].plot(gr, ug)

    # only set the tight layout on the first iteration
    if i == 0:
        plt.tight_layout()

    # save the frame
    fig.savefig(f"{frame_dir}/step{i}.png", dpi=500)

# get the list of frames
file_list = [f"{frame_dir}/step{i}.png" for i in range(len(glob(f"{frame_dir}/*")))]

# combine the frames into a video
clip = ImageSequenceClip(file_list, fps=4)
clip.write_gif("figures/color_redshift.gif")

In [ ]:
catalog = get_galaxy_data().sample(2_000, replace=False, random_state=0)
mags = catalog.to_numpy()[:, 1:]
colors = mags[:, :-1] - mags[:, 1:]

fig, axes = plt.subplots(1, 4, figsize=(7, 2), constrained_layout=True, dpi=150)

zmin = 0.75
zmax = 1.00
mask = (catalog["redshift"] > zmin) & (catalog["redshift"] < zmax)

fig.suptitle(f"{zmin:.2f} < z < {zmax:.2f}")
ticks = [0, 1, 2]

axes[0].scatter(colors[~mask][:, 1], colors[~mask][:, 0], s=1, marker=1, c="C0", zorder=0)
axes[0].scatter(colors[mask][:, 1], colors[mask][:, 0], s=1, marker=1, c="C3", zorder=10)
axes[0].set(ylabel="$u-g$", xlabel="$g-r$", xlim=(-0.5, 2.5), ylim=(-0.5, 2.5), xticks=ticks, yticks=ticks)

axes[1].scatter(colors[~mask][:, 2], colors[~mask][:, 1], s=1, marker=1, c="C0", zorder=0)
axes[1].scatter(colors[mask][:, 2], colors[mask][:, 1], s=1, marker=1, c="C3", zorder=10)
axes[1].set(ylabel="$g-r$", xlabel="$r-i$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

axes[2].scatter(colors[~mask][:, 3], colors[~mask][:, 2], s=1, marker=1, c="C0", zorder=0)
axes[2].scatter(colors[mask][:, 3], colors[mask][:, 2], s=1, marker=1, c="C3", zorder=10)
axes[2].set(ylabel="$r-i$", xlabel="$i-z$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

axes[3].scatter(colors[~mask][:, 4], colors[~mask][:, 3], s=1, marker=1, c="C0", zorder=0)
axes[3].scatter(colors[mask][:, 4], colors[mask][:, 3], s=1, marker=1, c="C3", zorder=10)
axes[3].set(ylabel="$i-z$", xlabel="$z-y$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

In [ ]:
catalog = get_galaxy_data().sample(2_000, replace=False, random_state=0)
mags = catalog.to_numpy()[:, 1:]
colors = mags[:, :-1] - mags[:, 1:]

fig, axes = plt.subplots(1, 4, figsize=(7, 2), constrained_layout=True, dpi=150)
ticks = [0, 1, 2]

zbins = np.arange(0, 2.25, 0.25)

# directory where individual frames are stored
frame_dir = "figures/color_space"

for i in range(len(zbins) - 1):
    zmin = zbins[i]
    zmax = zbins[i + 1]

    mask = (catalog["redshift"] > zmin) & (catalog["redshift"] < zmax)

    fig.suptitle(f"{zmin:.2f} < z < {zmax:.2f}")

    axes[0].clear()
    axes[0].scatter(colors[~mask][:, 1], colors[~mask][:, 0], s=1, marker=1, c="C0", zorder=0)
    axes[0].scatter(colors[mask][:, 1], colors[mask][:, 0], s=1, marker=1, c="C3", zorder=10)
    axes[0].set(ylabel="$u-g$", xlabel="$g-r$", xlim=(-0.5, 2.5), ylim=(-0.5, 2.5), xticks=ticks, yticks=ticks)

    axes[1].clear()
    axes[1].scatter(colors[~mask][:, 2], colors[~mask][:, 1], s=1, marker=1, c="C0", zorder=0)
    axes[1].scatter(colors[mask][:, 2], colors[mask][:, 1], s=1, marker=1, c="C3", zorder=10)
    axes[1].set(ylabel="$g-r$", xlabel="$r-i$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

    axes[2].clear()
    axes[2].scatter(colors[~mask][:, 3], colors[~mask][:, 2], s=1, marker=1, c="C0", zorder=0)
    axes[2].scatter(colors[mask][:, 3], colors[mask][:, 2], s=1, marker=1, c="C3", zorder=10)
    axes[2].set(ylabel="$r-i$", xlabel="$i-z$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

    axes[3].clear()
    axes[3].scatter(colors[~mask][:, 4], colors[~mask][:, 3], s=1, marker=1, c="C0", zorder=0)
    axes[3].scatter(colors[mask][:, 4], colors[mask][:, 3], s=1, marker=1, c="C3", zorder=10)
    axes[3].set(ylabel="$i-z$", xlabel="$z-y$", xlim=(-0.5, 2.0), ylim=(-0.5, 2.0), xticks=ticks, yticks=ticks)

    # save the frame
    fig.savefig(f"{frame_dir}/step{i}.png", dpi=500)


# get the list of frames
file_list = [f"{frame_dir}/step{i}.png" for i in range(len(glob(f"{frame_dir}/*")))]

# combine the frames into a video
clip = ImageSequenceClip(file_list, fps=0.5)
clip.write_gif("figures/color_space.gif")